# Get a single face from a video

In [150]:
#default_exp nb_01b

In [10]:
from fastai.vision import *
from kgl_deepfake.nb_00 import *
from kgl_deepfake.nb_01 import *
from IPython.display import HTML
import cv2
import pandas as pd
from facenet_pytorch import MTCNN
from nbdev.export import *

### Data

In [2]:
SOURCE = Path('../data/train_sample_videos/')

In [7]:
f = get_files(SOURCE, extensions=['.json'])[0]
annots = pd.read_json(f).T
annots.reset_index(inplace=True)
annots.rename({'index':'fname'}, axis=1, inplace=True)
annots.head()

,fname,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [9]:
fn = SOURCE/annots.fname[0]
fn

PosixPath('../data/train_sample_videos/aagfhgtpmv.mp4')

### Get the first detected face from a video

In [14]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(device=device, post_process=False)

In [38]:
resize = .5

In [95]:
#export
def get_first_face(detector, fn, resize=.5):
    '''
    Returns the first detected face from a video
    '''
    v_cap = cv2.VideoCapture(str(fn))
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    iframe, face = None, None
    for i in range(v_len):
        _ = v_cap.grab()
        success, frame = v_cap.retrieve()
        if not success: continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = PIL.Image.fromarray(frame)
        if resize is not None: frame = frame.resize([int(d * resize) for d in frame.size])
        face = detector(frame)
        if face is not None: 
            iframe = i
            break
        v_cap.release()
    return iframe, face

### Videos in which not a single face is detected by MTCNN

In [129]:
#export
def get_has_face(fnames, detector):
    if isinstance(fnames, (str, Path)): fnames = [fnames]
    res = []
    for i in progress_bar(range(len(fnames))):
        iframe, face = get_first_face(detector, fnames[i])
        res.append(True if iframe is not None else False)
    return res

In [130]:
fnames = [SOURCE/o for o in annots.fname]
hasface = get_has_face(fnames, detector)

In [131]:
len(hasface), len([o for o in hasface if o == False])

(400, 20)

In [134]:
annots_noface = annots[~np.array(hasface)]

In [135]:
annots_noface.shape

(20, 4)

In [136]:
fnames_noface = [SOURCE/o for o in annots_noface.fname]
labels = [f'{o.fname} {o.label}' for i, o in annots_noface.iterrows()]

In [137]:
HTML(html_vids(fnames_noface, titles=labels))

### `ItemList` that returns the first face from a video

In [138]:
#export
class VideoFaceList(ImageList):
    def __init__(self, *args, detector=None, resize=.5, device=None, **kwargs):
        if device is None: device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        if detector is None: detector = MTCNN(device=device, post_process=False)
        self.detector, self.resize = detector, resize
        super().__init__(*args, **kwargs)
    
    def get_face(self, fn:Path):
        iframe, face = get_first_face(self.detector, fn, self.resize)
        if iframe is None or face is None: raise Exception(f'No faces detected in {fn}')
        return iframe, face
    
    def open(self, fn:Path):
        iframe, face = self.get_face(fn)
        return Image(face / 255)

Let's test this, excluding those videos with no face detected.

In [139]:
annots[np.array(hasface)].shape

(380, 4)

In [140]:
annots_hasface = annots[np.array(hasface)]

In [141]:
src = VideoFaceList.from_df(df=annots_hasface, path=SOURCE, cols='fname').split_by_rand_pct()

In [142]:
%%time
src

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 34.1 µs


ItemLists;

Train: VideoFaceList (304 items)
Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160)
Path: ../data/train_sample_videos;

Valid: VideoFaceList (76 items)
Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160)
Path: ../data/train_sample_videos;

Test: None

In [143]:
%%time
data = src.label_from_df('label').databunch(bs=32, num_workers=0)

CPU times: user 15.1 s, sys: 2.1 s, total: 17.2 s
Wall time: 10.1 s


In [144]:
%%time
xb, yb = next(iter(data.train_dl))

CPU times: user 14.8 s, sys: 2.29 s, total: 17.1 s
Wall time: 10.5 s


In [145]:
xb.shape, yb.shape

(torch.Size([32, 3, 160, 160]), torch.Size([32]))

In [146]:
yb

tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [147]:
%%time 
xb, yb = next(iter(data.valid_dl))

CPU times: user 15.3 s, sys: 2.17 s, total: 17.5 s
Wall time: 10.1 s


In [148]:
xb.shape, yb.shape

(torch.Size([32, 3, 160, 160]), torch.Size([32]))

# -fin

In [149]:
notebook2script()

Converted 00_lookatdata.ipynb.
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
Converted 01_face_recog.ipynb.
Converted 01a_face_extraction.ipynb.
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
Converted 01b_get_one_face.ipynb.
Converted 02a_create_faceimage_dataset.ipynb.
Converted 02bis_Create_Dataset-Copy1.ipynb.
Converted 02bis_Create_Dataset.ipynb.
Converted 03_mesonet.ipynb.
Converted 04_Baseline_Classification-Copy1.ipynb.
Converted 04_Baseline_Classification.ipynb.
Converted 04_Classification-Copy1.ipynb.
Converted 04_Classification.ipynb.
Converted 04a_videolist.ipynb.
Converted 05_Class_Imbalance-Copy1.ipynb.
Converted 05_Class_I